# Test RLS on organization membership management (organization_user table)

In [ ]:
import sqlmodel as sqm
import pytest

# Test assets
from laminhub_rest.core.account import create_organization_account

# Test utils
from laminhub_rest.utils._test import create_test_auth, create_test_account
from laminhub_rest.connector import connect_hub_with_auth
from laminhub_rest.main import client
from laminhub_rest._clean_ci import clean_ci
from laminhub_rest.utils._id import base62

# Parametrize

In [ ]:
# Create account 1
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_1 = create_test_account(handle=auth_1["handle"], access_token=access_token_1)

# Create account 2
auth_2 = create_test_auth()
access_token_2 = auth_2["access_token"]
account_2 = create_test_account(handle=auth_2["handle"], access_token=access_token_2)

# Create account 3
auth_3 = create_test_auth()
access_token_3 = auth_3["access_token"]
account_3 = create_test_account(handle=auth_3["handle"], access_token=access_token_3)

# Create organization account
org_handle_1 = f"lamin.ci.org.{base62(8)}"
organization = create_organization_account(
    handle=org_handle_1, _access_token=access_token_1
)
hub = connect_hub_with_auth(access_token=access_token_1)
organization_id = (
    hub.table("account").select("id").eq("handle", org_handle_1).execute().data[0]["id"]
)

## Enable members to select other members

Add a member to the organization.

In [ ]:
client.post(
    f"organization/resources/members/{organization_id}/?user_id={account_2['id']}",
    headers={"authentication": f"Bearer {access_token_1}"},
)

Member should be able to select all members of the organization.

In [ ]:
response = client.get(
    f"organization/resources/members/{organization_id}",
    headers={"authentication": f"Bearer {access_token_2}"},
).json()

assert len(response) == 2

## Enable owners to add members

Non-owner member should not be able to add other members.

In [ ]:
with pytest.raises(Exception) as error:
    response = client.post(
        f"organization/resources/members/{organization_id}/?user_id={account_3['id']}",
        headers={"authentication": f"Bearer {access_token_2}"},
    ).json()

assert "new row violates row-level security policy" in error.value.message

## Enable owners to update members

Non-owner member should not be able to update other members.

In [ ]:
response = client.put(
    f"organization/resources/members/{organization_id}/?user_id={account_1['id']}&role=member",
    headers={"authentication": f"Bearer {access_token_2}"},
).json()

assert response == "member-not-updated"

## Enable owners to delete members

Non-owner member should not be able to delete other members.

In [ ]:
response = client.delete(
    f"organization/resources/members/{organization_id}/?user_id={account_1['id']}",
    headers={"authentication": f"Bearer {access_token_2}"},
).json()

assert response == "member-not-deleted"

## Clean up test assets

In [ ]:
clean_ci()